<a href="https://colab.research.google.com/github/souzajvp/educational/blob/main/An%C3%A1lises-geoespaciais/Primeiros-passos-geopandas/Preparando_os_dados_para_an%C3%A1lise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aqui faremos a preparação do conjunto de dados populacionais e de vacinação do paraná. Sendo assim, serão dois conjuntos:
1. Informações sobre a vacinação contra COVID-19 no Brasil;
2. População dos municípios do Paraná, segundo o senso de 2010.

1. Dados de vacinação de COVID-19, baixados no dia 17/07/2021 e disponibilizados no meu [GitHub](https://github.com/souzajvp/educational/tree/main/An%C3%A1lises-geoespaciais/Primeiros-passos-geopandas) - [Fonte](https://qsprod.saude.gov.br/extensions/DEMAS_C19Vacina/DEMAS_C19Vacina.html)

In [1]:
import pandas as pd

In [2]:
url = 'https://github.com/souzajvp/educational/blob/main/An%C3%A1lises-geoespaciais/Primeiros-passos-geopandas/vacinacao_covid_17_07_2021.xlsx?raw=true'
vac_covid = pd.read_excel(url)

2. População dos municípios do Paraná de acordo com o Censo de 2010, extraídas do [TabNet](http://tabnet.datasus.gov.br/cgi/deftohtm.exe?ibge/cnv/poppr.def) (DATASUS) e disponibilizadas no meu [GitHub](https://github.com/souzajvp/educational/tree/main/An%C3%A1lises-geoespaciais/Primeiros-passos-geopandas)

In [3]:
url = 'https://raw.githubusercontent.com/souzajvp/teaching-python/main/pop_parana_2010.csv'
pop_pr = pd.read_csv(url)

## Trabalhando os dados de vacinação

Filtrando para o estado do PR

In [4]:
vac_covid = vac_covid[vac_covid['UF'] == 'PR']
# Para alguns casos não há informação do Fabricante. Decidi remover
vac_covid = vac_covid.query('Fabricante != "-"')

Como meu objetivo é obter o total de pessoas completamente vacinadas, vou filtrar o banco de vacinação de acordo com o fabricante "JANSSEN". Essa vacina é a única que tem o regime de dose única

In [5]:
dose_unica = vac_covid.query('Fabricante == "JANSSEN"')
dose_dupla = vac_covid.query('Fabricante != "JANSSEN"')

In [6]:
# Aqui criei uma variável nova, que armazena a quantidade de doses aplicadas da vacina
# da JANSSEN
dose_unica['Doses_janssen'] = dose_unica['Doses Aplicadas']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Em seguida, posso agrupar e somar os dados de todas as outras vacinas de acordo com o município de aplicação.

Assim, terei a quantidade de primeiras e segundas doses aplicadas para cada município brasileiro.

In [7]:
dose_dupla_mun = dose_dupla.groupby('Cód. IBGE').sum().reset_index()

Para criar o conjunto de dados final de vacinação, vou usar o método `join` para agregar os dados de vacinação da Janssen de acordo com o município, sempre utilizando o Cód. IBGE como referência

In [8]:
vac_final = dose_dupla_mun.set_index('Cód. IBGE').join(dose_unica[['Cód. IBGE', 'Doses_janssen']].set_index('Cód. IBGE'))

Algumas cidades tem valor nulo para a vacina Janssen, para evitar problemas, vou preencher com 0

In [9]:
vac_final['Doses_janssen'].fillna(0, inplace=True)

In [10]:
vac_final['Imunizados'] = vac_final['Dose 2'] + vac_final['Doses_janssen']
vac_final = vac_final.reset_index()

Adicionando os dados populacionais aos dados de vacinação

In [11]:
vac_pr = pd.merge(vac_final, pop_pr, left_on='Cód. IBGE', right_on='ID_MUN_RES')

Agora, com as informações populacionais e vacinais, podemos calcular a porcentagem da população imunizada parcialmente e completamente

In [12]:
vac_pr['P_Comp_Imun'] =  (vac_pr['Imunizados'] / vac_pr['POP_2010']) *100
vac_pr['P_Parc_Imun'] =  (vac_pr['Dose 1'] / vac_pr['POP_2010']) *100

In [13]:
vac_pr['Cód. IBGE'] = vac_pr['Cód. IBGE'].astype(str)

Salvando o arquivo contendo dados populacionais e de vacinação do PR

In [14]:
vac_pr.to_csv('vacinacao_populacao_parana.csv',index=False)